In [12]:
import polars as pl
import os
from openai import OpenAI
from pathlib import Path

In [13]:
client = OpenAI(api_key='sk-proj-HSrHGyOHtEDtr9loLzRjT3BlbkFJTUFaqCLYDEs3B7qpBz7z')

In [14]:
all_transcripts = []

In [15]:
split_audio_path = Path().cwd().parent.joinpath('data/split_audio/')

In [16]:
def clean_filenames():
    filenames = os.listdir(split_audio_path)
    for filename in filenames:
        os.rename(os.path.join(split_audio_path, filename), os.path.join(split_audio_path, filename.replace(' ', '_')))

In [17]:
clean_filenames()

In [18]:
def transcribe(audio_import):
    transcription = client.audio.transcriptions.create(
        model="whisper-1", 
        file=audio_import,
        response_format="verbose_json",
        timestamp_granularities=["segment"]
  )
    return transcription

In [19]:
split_audio_filenames = os.listdir(split_audio_path)

In [20]:
def batch_transcribe():
    for file in sorted(split_audio_filenames):
        file_path = Path().cwd().parent.joinpath(f'data/split_audio/{file}') 

        #file_path = f"{split_audio_path}/{file}"

        transcription = transcribe(file_path)

        transcription_str = str(transcription.text)

        transcription_sent = transcription_str.split('. ')

        station_callsign = file.split("_",1)[0]

        tt_dict = {
            "callsign": station_callsign, 
            "file_name": file, 
            "transcription": transcription_sent, 
            }

        all_transcripts.append(tt_dict)

        print(f"Finished transcribing and translating {file}")


In [21]:
batch_transcribe()

Finished transcribing and translating 2024-05-07_00-21-36_000.mp3
Finished transcribing and translating 2024-05-07_00-21-36_001.mp3
Finished transcribing and translating 2024-05-07_00-21-36_002.mp3
Finished transcribing and translating 2024-05-07_00-21-36_003.mp3
Finished transcribing and translating 2024-05-07_00-21-36_004.mp3
Finished transcribing and translating 2024-05-07_00-21-36_005.mp3


KeyboardInterrupt: 

In [22]:
all_transcripts

[{'callsign': '2024-05-07',
  'file_name': '2024-05-07_00-21-36_000.mp3',
  'transcription': ['Otavola es un tipo, como te lo dije aquel día, que no quiero decir nombre, porque son gente que ya pasaron la página esa, pero mucho que coquetearon con la Revolución Cubana, iban a Cuba, llevaron a artistas importantes, invitados, denominaron a Díaz-Taner como presidente de Cuba',
   'Por suerte, ya eso pasó',
   'Gracias, Otavola, pues si no les pone el dedo como se lo puso el que quería quitar esta residencia, a lo mejor no hubieran cambiado, pero bueno, ya eso es agua pasada',
   'Lo de Caído no lo sabía',
   'Yo creo que hiciste bien decir la precisión',
   'No, no, es que no',
   'Oye, una cosa no tiene que dar más',
   'Este es el trabajo de nosotros',
   'Con qué licencia y con qué? Un hombre que se le ha tratado muy bien aquí para ponerme el dado malo a mí aquí',
   'De que esto, de que lo otro no',
   'Eso no se hace',
   'Eso no se lo hago yo a nadie',
   'No, no, no, eso no se lo 

In [23]:
transcript_df = pl.DataFrame(all_transcripts)

In [24]:
transcript_df

shape: (6, 3)
┌────────────┬─────────────────────────────┬─────────────────────────────────┐
│ callsign   ┆ file_name                   ┆ transcription                   │
│ ---        ┆ ---                         ┆ ---                             │
│ str        ┆ str                         ┆ list[str]                       │
╞════════════╪═════════════════════════════╪═════════════════════════════════╡
│ 2024-05-07 ┆ 2024-05-07_00-21-36_000.mp3 ┆ ["Otavola es un tipo, como te … │
│ 2024-05-07 ┆ 2024-05-07_00-21-36_001.mp3 ┆ ["How about a quarter to ten? … │
│ 2024-05-07 ┆ 2024-05-07_00-21-36_002.mp3 ┆ ["Ahora, de ahora en adelante,… │
│ 2024-05-07 ┆ 2024-05-07_00-21-36_003.mp3 ┆ ["Votación, sube la música", "… │
│ 2024-05-07 ┆ 2024-05-07_00-21-36_004.mp3 ┆ ["Oh, oh, who's this? Da, di, … │
│ 2024-05-07 ┆ 2024-05-07_00-21-36_005.mp3 ┆ ["la producción, y eso hace qu… │
└────────────┴─────────────────────────────┴─────────────────────────────────┘

In [25]:
json_name = 'transcript.json'

In [26]:
output_json_path = Path().cwd().parent.joinpath(f'data/output_json/{json_name}') 


In [28]:
transcript_df.write_json(output_json_path)